In [1]:
!pip install numpy==1.23.1
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Defaulting to user installation because normal site-packages is not writeable
1.23.1


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import re
from sklearn.preprocessing import LabelEncoder

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
data = pd.read_pickle('dataset_small.pkl')
data # 796867

,상호명,상권업종대분류명,상권업종중분류명
0,엠마스키친,음식,서양식
1,팽성농산물센터,소매,식료품 소매
2,역전할머니맥주평택궁리점,음식,주점
3,혜윰건축사사무소,과학·기술,기술 서비스
4,양촌리민물장어,음식,한식
...,...,...,...
796862,다리미,소매,섬유·의복·신발 소매
796863,청하중화요리,음식,중식
796864,대칭점,음식,비알코올
796865,나베르떼헤어,수리·개인,이용·미용


In [9]:
encoder = LabelEncoder()
label_dicts = {} 

for column in ['상권업종대분류명',	'상권업종중분류명']:
  encoder.fit(data[column])
  label_dict = dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
  data[column] = encoder.transform(data[column])
  label_dicts[column] = label_dict  # 해당 컬럼의 딕셔너리를 저장

data.head()

,상호명,상권업종대분류명,상권업종중분류명
0,엠마스키친,9,33
1,팽성농산물센터,4,41
2,역전할머니맥주평택궁리점,9,66
3,혜윰건축사사무소,0,8
4,양촌리민물장어,9,73


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
class BERTClassifier1(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 10, # big
                 dr_rate=None,
                 params=None):
        super(BERTClassifier1, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
class BERTClassifier2(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 75, # mid
                 dr_rate=None,
                 params=None):
        super(BERTClassifier2, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
def prepare_data(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=64,
        return_token_type_ids=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    token_type_ids = encoding['token_type_ids']
    valid_length = torch.tensor([torch.sum(attention_mask[0])], dtype=torch.long)

    return input_ids, attention_mask, token_type_ids, valid_length

In [ ]:
def predict(model, input_ids, attention_mask, token_type_ids, valid_length):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    valid_length = valid_length.to(device)

    with torch.no_grad():
        outputs = model(input_ids, valid_length, token_type_ids)
        probabilities = torch.softmax(outputs, dim=1) 
        predicted_class = torch.argmax(probabilities, dim=1)
        return predicted_class

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
df1 = data[['상호명', '상권업종대분류명']] # big
df2 = data[['상호명', '상권업종중분류명']] # mid

In [18]:
train1, test1 = train_test_split(df1, test_size=0.25, shuffle = True, random_state=0)
train2, test2 = train_test_split(df2, test_size=0.25, shuffle = True, random_state=0)

train1 = [[i, str(j)] for i, j in zip(train1['상호명'], train1['상권업종대분류명'])]
train2 = [[i, str(j)] for i, j in zip(train2['상호명'], train2['상권업종중분류명'])]

test1 = [[i, str(j)] for i, j in zip(test1['상호명'], test1['상권업종대분류명'])]
test2 = [[i, str(j)] for i, j in zip(test2['상호명'], test2['상권업종중분류명'])]

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [19]:
# big
train_set_data1 = BERTDataset(train1, 0, 1, tok, vocab, max_len, True, False)
test_set_data1 = BERTDataset(test1, 0, 1, tok, vocab, max_len, True, False)
train_dataloader1 = torch.utils.data.DataLoader(train_set_data1, batch_size=batch_size, num_workers=2)
test_dataloader1 = torch.utils.data.DataLoader(test_set_data1, batch_size=batch_size, num_workers=2)

# mid
train_set_data2 = BERTDataset(train2, 0, 1, tok, vocab, max_len, True, False)
test_set_data2 = BERTDataset(test2, 0, 1, tok, vocab, max_len, True, False)
train_dataloader2 = torch.utils.data.DataLoader(train_set_data2, batch_size=batch_size, num_workers=2)
test_dataloader2 = torch.utils.data.DataLoader(test_set_data2, batch_size=batch_size, num_workers=2)

# train

In [20]:
# train - big
model1 = BERTClassifier1(bertmodel, dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model1.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model1.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader1) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model1.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader1)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader1)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [21]:
torch.save(model1, 'model/model1.pth')

In [22]:
# train - mid
model2 = BERTClassifier2(bertmodel, dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader2) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model2.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader2)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model2.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader2)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-22-0d7298afdfa5>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader2)):


  0%|          | 0/9339 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 4.3969502449035645 train acc 0.015625
epoch 1 batch id 201 loss 4.121189117431641 train acc 0.036069651741293535
epoch 1 batch id 401 loss 3.9892735481262207 train acc 0.06729270573566085
epoch 1 batch id 601 loss 3.8084254264831543 train acc 0.08353265391014975
epoch 1 batch id 801 loss 3.5047385692596436 train acc 0.11060393258426966
epoch 1 batch id 1001 loss 2.8618643283843994 train acc 0.14434003496503497
epoch 1 batch id 1201 loss 2.813129186630249 train acc 0.18230901332223148
epoch 1 batch id 1401 loss 2.6014840602874756 train acc 0.22003256602426838
epoch 1 batch id 1601 loss 2.2721054553985596 train acc 0.25246915990006247
epoch 1 batch id 1801 loss 2.6570253372192383 train acc 0.28064269850083284
epoch 1 batch id 2001 loss 2.1423017978668213 train acc 0.30576899050474765
epoch 1 batch id 2201 loss 1.9793694019317627 train acc 0.32788363243980007
epoch 1 batch id 2401 loss 1.7088935375213623 train acc 0.3480125468554769
epoch 1 batch id 2601 loss 2.215

<ipython-input-22-0d7298afdfa5>:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):


  0%|          | 0/3113 [00:00<?, ?it/s]

...

epoch 4 test acc 0.679732212512374


  0%|          | 0/9339 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.0855484008789062 train acc 0.6875
epoch 5 batch id 201 loss 1.0484747886657715 train acc 0.7273009950248757
epoch 5 batch id 401 loss 1.0955142974853516 train acc 0.7303615960099751
epoch 5 batch id 601 loss 1.2289292812347412 train acc 0.7310732113144759
epoch 5 batch id 801 loss 1.179101824760437 train acc 0.7308832709113608
epoch 5 batch id 1001 loss 1.1172690391540527 train acc 0.7301448551448552
epoch 5 batch id 1201 loss 0.8870865106582642 train acc 0.7292360532889259
epoch 5 batch id 1401 loss 0.9987618327140808 train acc 0.729958511777302
epoch 5 batch id 1601 loss 1.1202068328857422 train acc 0.7304711898813242
epoch 5 batch id 1801 loss 1.1076183319091797 train acc 0.730097862298723
epoch 5 batch id 2001 loss 0.6442956924438477 train acc 0.730181784107946
epoch 5 batch id 2201 loss 0.818857729434967 train acc 0.7300232848705134
epoch 5 batch id 2401 loss 0.8893573880195618 train acc 0.7304378384006663
epoch 5 batch id 2601 loss 0.9617353677749634 tra

  0%|          | 0/3113 [00:00<?, ?it/s]

epoch 5 test acc 0.6805051807102539


In [23]:
torch.save(model2, 'model/model2.pth')